In [31]:
import numpy as np
import pandas as pd

DATE = '2020_04_06'

In [32]:
data = pd.read_csv(f'countermeasures_features_{DATE}.csv').drop(columns='Unnamed: 0').fillna(0)


In [33]:




REPORTCOLS = {'Widespread mask usage (over 75%)':('Mask wearing',lambda x: x>=75),
              'Confirmed case isolation':('Symptomatic isolation - targeted',lambda x: x>=1),
              'Contact isolation':('Asymptomatic isolation - targeted',lambda x: x>=1),
              'Gatherings limited to 10 or less':('Gatherings banned',lambda x: (x>1) and (x<=10)),
              'Gatherings limited to 100':('Gatherings banned',lambda x: (x>10) and (x<=100)),
              'Gatherings limited to 500':('Gatherings banned',lambda x: (x>100) and (x<=500)),
              'Some businesses suspended':('Activity cancellation',lambda x:(x>=1) and (x<=2)),
              'Many businesses suspended':('Nonessential business suspension',lambda x: x>=1),
              'Schools and universities closed':('School closure',lambda x: x>=1),
              'Social distancing advised':('Public interaction reduction',lambda x: (x>=1) and (x<3)),
              'Staying home advised':('Public interaction reduction',lambda x: x>=3),
              '1 - 10 000 tests':('Testing',lambda x: (x>=1) and (x<=10000)),
              '10 000 - 100 000 tests':('Testing',lambda x: (x>10000) and (x<=1e5)),
              '100 000+ tests':('Testing',lambda x: x>1e5),
              'General curfew':('Asymptomatic isolation - blanket',lambda x: (x>=1) and (x<2)),
              'General strict isolation':('Asymptomatic isolation - blanket',lambda x: x>=2)}

for rc, (vc, f) in REPORTCOLS.items():
    data[rc] = data[vc].apply(f).astype(int)
    
outdata = data[list(REPORTCOLS.keys())+['Country','Date']]

outdata.to_csv(f'interventions_visualisation_boolean_{DATE}.csv')



KeyError: 'Public interaction reduction'

In [34]:
def gatherings(n):
    if n == np.nan:
        return 0
    return 1/(1+np.exp(3-np.log(n)))


def curfew(n):
    if n < 1:
        return 0
    elif n<2:
        return 0.5
    return 1

def norm(maximum):
    def f(n):
        return n/maximum
    return f
    
def nj2us(s):
    if s=='US:New Jersey':
        return 'United States'
    return s

REPORTCOLS1 = {'Gatherings limited to':('Gatherings banned',gatherings),
              'Business suspended':('Nonessential business suspension',norm(1)),
              'Schools and universities closed':('School closure',lambda x: x>=1),
              'General curfew':('Asymptomatic isolation - blanket',curfew),
              'Public information':('Public education and incentives',norm(4)),
              'Healthcare specialisation':('Healthcare specialisation',norm(7)),
              'Public hygiene':('Public interaction and hygiene',norm(12))}

data0 = data.copy()

US_STATES = data0['Country'][data0['Country'].str.startswith('US:')].unique()

data0['Country'] = data0['Country'].apply(nj2us)
data0.loc[data['Country']=='United States',
                  data0.columns[:-1]] = data0.loc[data['Country'].isin(US_STATES),:].groupby('Date').mean()

data0 = data0[~data0['Country'].str.startswith('US:')]

for rc, (vc, f) in REPORTCOLS1.items():
    data0[rc] = data0[vc].apply(f).astype(float)
    
outdata = data0[list(REPORTCOLS1.keys())+['Country','Date']]
outdata.to_csv(f'interventions_model_zeroto1_{DATE}.csv')